In [1]:
import pandas as pd
import numpy as np
import re
import zipfile
import os
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10054] An existing connection was forcibly
[nltk_data]     closed by the remote host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>
[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [2]:
df = pd.read_csv('final_hateXplain.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20109 entries, 0 to 20108
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comment             20109 non-null  object
 1   label               20109 non-null  object
 2   Race                20109 non-null  object
 3   Religion            20109 non-null  object
 4   Gender              20109 non-null  object
 5   Sexual Orientation  20109 non-null  object
 6   Miscellaneous       3533 non-null   object
dtypes: object(7)
memory usage: 1.1+ MB


In [4]:
df.drop(columns=['Miscellaneous'], inplace=True)

In [5]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive,No_race,Nonreligious,No_gender,No_orientation
2,1 0 yrs <number> white women raped by niggers ...,hatespeech,African,Nonreligious,No_gender,No_orientation
3,1 2 h ago ching chong accepted your friend req...,offensive,Asian,Nonreligious,No_gender,No_orientation
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation


In [6]:
df['comment'] = df['comment'].apply(lambda x: nltk.word_tokenize(x))

In [7]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,"[0, u0, lmao, wow, fuck, you, too, 😂, 😂]",normal,No_race,Nonreligious,No_gender,No_orientation
1,"[1, 0, th, floor, maybe, wow, cnn, with, the, ...",offensive,No_race,Nonreligious,No_gender,No_orientation
2,"[1, 0, yrs, <, number, >, white, women, raped,...",hatespeech,African,Nonreligious,No_gender,No_orientation
3,"[1, 2, h, ago, ching, chong, accepted, your, f...",offensive,Asian,Nonreligious,No_gender,No_orientation
4,"[1, 8, th, century, mayhem, and, lawlessness, ...",normal,No_race,Nonreligious,No_gender,No_orientation


In [8]:
def remove_stopwords(tokens_list):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens_list if word not in stop_words]
    
    return filtered_tokens

df['comment'] = df['comment'].apply(lambda tokens_list: remove_stopwords(tokens_list))

In [9]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,"[0, u0, lmao, wow, fuck, 😂, 😂]",normal,No_race,Nonreligious,No_gender,No_orientation
1,"[1, 0, th, floor, maybe, wow, cnn, fakenews, b...",offensive,No_race,Nonreligious,No_gender,No_orientation
2,"[1, 0, yrs, <, number, >, white, women, raped,...",hatespeech,African,Nonreligious,No_gender,No_orientation
3,"[1, 2, h, ago, ching, chong, accepted, friend,...",offensive,Asian,Nonreligious,No_gender,No_orientation
4,"[1, 8, th, century, mayhem, lawlessness, nothi...",normal,No_race,Nonreligious,No_gender,No_orientation


In [10]:
def remove_digits(tokens_list):
    token_filtered = []
    for token in tokens_list:
        result = ''.join([i for i in token if not i.isdigit()])
        if result: 
            token_filtered.append(result)
    
    return token_filtered

df['comment'] = df['comment'].apply(lambda tokens:remove_digits(tokens))

In [11]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,"[u, lmao, wow, fuck, 😂, 😂]",normal,No_race,Nonreligious,No_gender,No_orientation
1,"[th, floor, maybe, wow, cnn, fakenews, body, c...",offensive,No_race,Nonreligious,No_gender,No_orientation
2,"[yrs, <, number, >, white, women, raped, nigge...",hatespeech,African,Nonreligious,No_gender,No_orientation
3,"[h, ago, ching, chong, accepted, friend, reque...",offensive,Asian,Nonreligious,No_gender,No_orientation
4,"[th, century, mayhem, lawlessness, nothing, lo...",normal,No_race,Nonreligious,No_gender,No_orientation


In [12]:
def remove_special_characters(tokens_list):
    result = [''.join(letter for letter in token if letter.isalnum()) for token in tokens_list]
    
    return result

df['comment'] = df['comment'].apply(lambda x:remove_special_characters(x))

In [13]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,"[u, lmao, wow, fuck, , ]",normal,No_race,Nonreligious,No_gender,No_orientation
1,"[th, floor, maybe, wow, cnn, fakenews, body, c...",offensive,No_race,Nonreligious,No_gender,No_orientation
2,"[yrs, , number, , white, women, raped, niggers...",hatespeech,African,Nonreligious,No_gender,No_orientation
3,"[h, ago, ching, chong, accepted, friend, reque...",offensive,Asian,Nonreligious,No_gender,No_orientation
4,"[th, century, mayhem, lawlessness, nothing, lo...",normal,No_race,Nonreligious,No_gender,No_orientation


In [14]:
def remove_empty_string(tokens_list):
    result = [token for token in tokens_list if token]
    
    return result

df['comment'] = df['comment'].apply(lambda x: remove_empty_string(x))

In [15]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,"[u, lmao, wow, fuck]",normal,No_race,Nonreligious,No_gender,No_orientation
1,"[th, floor, maybe, wow, cnn, fakenews, body, c...",offensive,No_race,Nonreligious,No_gender,No_orientation
2,"[yrs, number, white, women, raped, niggers, nu...",hatespeech,African,Nonreligious,No_gender,No_orientation
3,"[h, ago, ching, chong, accepted, friend, request]",offensive,Asian,Nonreligious,No_gender,No_orientation
4,"[th, century, mayhem, lawlessness, nothing, lo...",normal,No_race,Nonreligious,No_gender,No_orientation


In [16]:
df['comment'] = df['comment'].str.join(" ")

In [17]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,u lmao wow fuck,normal,No_race,Nonreligious,No_gender,No_orientation
1,th floor maybe wow cnn fakenews body count goi...,offensive,No_race,Nonreligious,No_gender,No_orientation
2,yrs number white women raped niggers number ne...,hatespeech,African,Nonreligious,No_gender,No_orientation
3,h ago ching chong accepted friend request,offensive,Asian,Nonreligious,No_gender,No_orientation
4,th century mayhem lawlessness nothing loud mou...,normal,No_race,Nonreligious,No_gender,No_orientation


In [18]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [19]:
lemmatizer = WordNetLemmatizer()
def lemmatize_sentence(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    for i in tokens:
        lemmatized_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
        
    return lemmatized_sentence
    
df['comment'] = df['comment'].apply(lambda x:lemmatize_sentence(x))

In [20]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,"[u, lmao, wow, fuck]",normal,No_race,Nonreligious,No_gender,No_orientation
1,"[th, floor, maybe, wow, cnn, fakenews, body, c...",offensive,No_race,Nonreligious,No_gender,No_orientation
2,"[yrs, number, white, woman, rap, nigger, numbe...",hatespeech,African,Nonreligious,No_gender,No_orientation
3,"[h, ago, ching, chong, accept, friend, request]",offensive,Asian,Nonreligious,No_gender,No_orientation
4,"[th, century, mayhem, lawlessness, nothing, lo...",normal,No_race,Nonreligious,No_gender,No_orientation


In [21]:
df['comment'] = df['comment'].str.join(" ")

In [22]:
df.head()

,comment,label,Race,Religion,Gender,Sexual Orientation
0,u lmao wow fuck,normal,No_race,Nonreligious,No_gender,No_orientation
1,th floor maybe wow cnn fakenews body count go ...,offensive,No_race,Nonreligious,No_gender,No_orientation
2,yrs number white woman rap nigger number negre...,hatespeech,African,Nonreligious,No_gender,No_orientation
3,h ago ching chong accept friend request,offensive,Asian,Nonreligious,No_gender,No_orientation
4,th century mayhem lawlessness nothing loud mou...,normal,No_race,Nonreligious,No_gender,No_orientation


In [23]:
df['label'].value_counts()

label
normal        7818
hatespeech    6484
offensive     5807
Name: count, dtype: int64

In [24]:
X = df['comment']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2054)

In [25]:
X_train.shape

(17092,)

In [26]:
X_test.shape

(3017,)

In [27]:
clf1 = Pipeline([
    ('vectorisor_tfidf',TfidfVectorizer()),
    ('lr',LogisticRegression())
])

In [28]:
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

  hatespeech       0.70      0.70      0.70       979
      normal       0.77      0.65      0.71      1397
   offensive       0.40      0.54      0.46       641

    accuracy                           0.64      3017
   macro avg       0.62      0.63      0.62      3017
weighted avg       0.67      0.64      0.65      3017



C:\Users\maina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
clf2 = Pipeline([
    ('vectorisor_tfidf',TfidfVectorizer()),
    ('rfc',RandomForestClassifier(n_estimators=100, random_state=42))
])

In [30]:
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

  hatespeech       0.69      0.72      0.70       948
      normal       0.80      0.61      0.69      1550
   offensive       0.35      0.58      0.44       519

    accuracy                           0.64      3017
   macro avg       0.61      0.64      0.61      3017
weighted avg       0.69      0.64      0.65      3017



In [31]:
clf3 = Pipeline([
    ('vectorisor_tfidf',TfidfVectorizer()),
    ('svc',SVC(probability=True, kernel='linear'))
])

In [32]:
clf3.fit(X_train, y_train)
y_pred = clf3.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

  hatespeech       0.70      0.72      0.71       956
      normal       0.75      0.66      0.70      1349
   offensive       0.43      0.51      0.46       712

    accuracy                           0.64      3017
   macro avg       0.62      0.63      0.62      3017
weighted avg       0.66      0.64      0.65      3017



In [33]:
voting_clf = VotingClassifier(estimators=[
        ('lr', clf1), 
        ('rf', clf2), 
        ('svc', clf3)], voting='hard')
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(classification_report(y_pred, y_test))

C:\Users\maina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

  hatespeech       0.71      0.71      0.71       981
      normal       0.77      0.65      0.71      1400
   offensive       0.40      0.54      0.46       636

    accuracy                           0.65      3017
   macro avg       0.63      0.64      0.63      3017
weighted avg       0.67      0.65      0.66      3017



In [34]:
model = voting_clf # final model

In [35]:
joblib.dump(model, 'model.joblib')
mdl = joblib.load('model.joblib')

In [36]:
corpus = [
    "I love spending time with my family on weekends.",
    "People from that country are ruining everything, they should be kicked out.",
    "You're so stupid, I can't believe you messed that up again!"
]

print(mdl.predict(corpus))

['normal' 'normal' 'hatespeech']


In [37]:
text_input = widgets.Text(
    value='',
    placeholder='Type something...',
    description='Input:',
    disabled=False
)

submit_button = widgets.Button(description="Submit")

output = widgets.Output()

def process_input(b):
    with output:
        clear_output()
        
        input_text = text_input.value
        result = f"You entered: {input_text}"
        print(f"Entered input: {mdl.predict([result])[0]}")

submit_button.on_click(process_input)
display(text_input, submit_button, output)

Text(value='', description='Input:', placeholder='Type something...')

Button(description='Submit', style=ButtonStyle())

Output()